# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится. Ее я делал раннее в другом проекте

## Откройте и изучите файл

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [20]:
data = pd.read_csv('/datasets/users_behavior.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB



<div class="alert-warning"> 
<b>Комментарий ревьюера 💡</b> 

.info() правильнее


In [21]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [22]:
data.shape

(3214, 5)

In [23]:
data.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


С данными все хорошо, нет нигде никаких особых отклонений и выбросов, is_ultra - наша таргет переменная


<div class="alert-warning"> 
<b>Комментарий ревьюера 💡</b> 

Стоит еще выводить .head()

## Разбейте данные на выборки

In [24]:
train, data1 = train_test_split(data, test_size=0.4, random_state=12345)
target_train = train['is_ultra']
features_train = train.drop(['is_ultra'] , axis=1)
target = data1['is_ultra']
features = data1.drop(['is_ultra'] , axis=1)
features_test, features_valid, target_test, target_valid = train_test_split(
    features, target, test_size=0.5, random_state=12345)
print(features_train.shape, features_valid.shape, features_test.shape, target_train.shape, target_valid.shape, target_test.shape)

(1928, 4) (643, 4) (643, 4) (1928,) (643,) (643,)


<div class="alert-warning"> 
<b>Комментарий ревьюера 💡 </b>

Стоит еще выводить размеры выборок для проверки, все ли прошло успешно. Например, с помощью shape

## Исследуйте модели

In [25]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
predict_valid = model.predict(features_valid)
print(accuracy_score(target_valid, predict_valid))

0.6842923794712286


У логистической регрессии как я и ожидал очень низкая степень точности

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡 Вторая итерация </b> 
    
accuracy - это не точность, а доля правильных ответов. Точность - другая метрика классификации, precision

In [26]:

for depth in range(1, 16, 1):
    for types in ['gini', 'entropy']:
            model = DecisionTreeClassifier(max_depth=depth, random_state=12345, criterion = types)
            model.fit(features_train, target_train)
            predict_valid = model.predict(features_valid)
            result = accuracy_score(target_valid, predict_valid)
            if result > best_result:
                best_criteria = types
                best_model = model
                best_result = result
                best_depth = depth
                best_est = est
                best_weight = weight_type
print('accuracy = ',best_result, 'depth = ',best_depth, 'criterion = ', best_criteria)

accuracy =  0.8149300155520995 depth =  6 criterion =  gini


<div class="alert-danger"> 
<b>Комментарий ревьюера ❌</b> 

Нет никакого подбора гиперпараметров, пункт "Исследуйте качество разных моделей, меняя гиперпараметры" не выполнен

С первой модели получилось достаточно высока степень accuracy, но мы можем сделать больше, построим лес!


Если я правильно понял, то гиперпараметры это например глубина дерева и количество деревьев, я их все перебрал в цикле, разве это не считается за пересмотр нескольких гиперпараметров?

<div class="alert-danger"> 
<b>Комментарий ревьюера ❌ Вторая итерация </b> 

Для DecisionTreeClassifier ты ничего не перебираешь. Два гиперпараметра для Random Forest недостаточно. Минимальный критерий принятия проекта - это рассмотрение хотя бы трех гиперпараметров для одной модели

In [29]:
best_result = 0.1
for est in range(1, 50, 1):
    for depth in range(2, 20, 1):
        for types in ['gini', 'entropy']:
            for weight_type in ['balanced', 'balanced_subsample', None]:
                model = RandomForestClassifier(n_estimators = est, max_depth=depth, random_state=12345, criterion = types, class_weight = weight_type)
                model.fit(features_train, target_train)
                predict_valid = model.predict(features_valid)
                result = accuracy_score(target_valid, predict_valid)
                if result > best_result:
                    best_criteria = types
                    best_model = model
                    best_result = result
                    best_depth = depth
                    best_est = est
                    best_weight = weight_type
print('accuracy = ',best_result, 'depth = ',best_depth, 'n of trees = ', best_est, 'criterion = ', best_criteria, 'weight type = ', best_weight)

accuracy =  0.8242612752721618 depth =  9 n of trees =  13 criterion =  entropy weight type =  None


<div class="alert-danger"> 
<b>Комментарий ревьюера ❌</b> 

Аналогично с прошлым комментарием, у леса можно много чего глянуть

Стало чуть лучше - 0.825, проверим на тестовой выборке

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡 </b>

Рассмотрены только две модели. Можно побольше использовать: например, логистическую регрессию

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡 Ко всему пукнту </b> 
    
Для более красивого оформления кода можно использовать f-strings. Подробнее про них почитать можешь здесь или на любом другом сайте: https://python-scripts.com/f-strings

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 Третья итерация </b>

Теперь здорово

## Проверьте модель на тестовой выборке

In [30]:
predict_test = best_model.predict(features_test)
print(accuracy_score(target_test, predict_test))

0.8040435458786936


Модель на тестовой выборке имеет чуть менее высокую точность, но она все равно достаточно высокая - 0,80
Можно было сделать еще модель логистической регрессии, но она зачастую выдает меньше, чем случайный лес, поэтому я предпочту его

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>

Модель проверена на тесте

## (бонус) Проверьте модели на адекватность

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡 </b>

Можно посмотреть на баланс классов: например, если у нас 90 процентов таргета это "1", а остальные 10 - "0", то мы можем тупо выдавать каждый раз цифру 1 и получать метрику 90 %. Любая модель, которая будет выдавать качество ниже этого - плохая/неадекватная

<div class="alert-danger"> 
<b>Комментарий ревьюера ❌</b>

Хороший проект, тобою проделана большая работа, ты молодец!

Сперва похвалю за наличие везде промежуточных выводов, комментариев. Хороший код, показываешь навыки владения sklearn. Все требуемые пункты в задании в целом выполнены. Оставил пару желтых комментариев, думаю, это сможет улучшить твой проект. Также можно добавить визуализацию в проект: посмотреть на графиках, как влияет перебор параметров на результат нашей метрики

В этом проекте большое внимание уделяется подбору гиперпараметров, но это можно автоматизировать! Советую почитать на досуге про GridsearchCV и Optuna, будет полезно
    
Также отправляю ссылки на документации некоторых моделей для классификации: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html, https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html, https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
    
Что обязательно стоит доработать: посмотреть побольше гиперпараметров у решающего дерева и случайного леса
    
Из курсов и дополнительной информации советую Yandex ML Handbook (https://academy.yandex.ru/handbook/ml) и курсы Евгения Соколова на ФКН (Машинное обучение - 1 с математикой https://github.com/esokolov/ml-course-hse и Основы машинного обучения без глубокой математики https://github.com/hse-ds/iad-intro-ds) - там можешь получить дополнительную информацию, более глубокую, возможно. http://wiki.cs.hse.ru/Заглавная_страница - тут можешь глянуть записи семинаров и лекций и другие материалы соответствующих курсов. 
    
Буду ждать доработок, удачи!

<div class="alert-danger"> 
<b>Комментарий ревьюера ❌ Вторая итерация </b>

Такие же ошибки

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 Третья итерация </b>

Все критические ошибки исправлены, проект стал еще лучше, принимаю. Успехов в дальнейшей учебе!

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
